In [1]:
import tensorflow as tf
import pandas as pd

file_path = "Processed_Data/Mapped_Dataset.csv"
batch_size = 512
train_ratio = 0.8

# Load file CSV dưới dạng tf.data.Dataset
dataset = tf.data.experimental.make_csv_dataset(
    file_path, 
    batch_size=batch_size, 
    label_name="label",  # Thay bằng tên cột label thật trong CSV
    num_epochs=1, 
    shuffle=True
)

# Tiền xử lý dữ liệu
def preprocess(features, label):
    label = tf.one_hot(label, depth=10)  # One-hot encoding
    features = tf.stack(list(features.values()), axis=-1)  # Chuyển các cột thành tensor
    features = tf.reshape(features, (-1, 46, 1))  # Định dạng input
    return features, label

dataset = dataset.map(preprocess)

# Chia train/test chính xác
dataset = dataset.shuffle(10000)  # Shuffle toàn bộ trước khi chia
train_size = int(train_ratio * sum(1 for _ in dataset))  # Số batch train
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Batch và prefetch để tối ưu tốc độ
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Kiểm tra dữ liệu
for X_batch, y_batch in train_dataset.take(1):
    print("Train batch shape:", X_batch.shape, y_batch.shape)


KeyboardInterrupt: 

In [ ]:
model = Sequential([
    Conv1D(64, kernel_size=3, activation="relu", input_shape=(46, 1)),  
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model.fit(train_dataset, epochs=10, validation_data=test_dataset)